In [1]:
import pandas as pd
import pyodbc
from fuzzywuzzy import fuzz
import datetime as dt
import datamart

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [22]:
import pandas as pd
import pyodbc
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(s)
npi_query= \
    """ 
    SELECT DISTINCT 
    I.ME,
    K.KEY_VAL AS NPI_NUMBER
    FROM 
    AMAEDW.PARTY_KEY K, AMAEDW.PARTY_ID_TO_ME_VW I
    WHERE  
    K.KEY_TYPE_ID = 38
    AND
    K.ACTIVE_IND = 'Y'
    AND
    I.PARTY_ID = K.PARTY_ID
    """
me_npi = pd.read_sql(con=AMAEDW, sql=npi_query)
me_npi

,ME,NPI_NUMBER
0,51708940261,1003000126
1,02012030021,1003000134
2,70416990113,1003000142
3,85802000010,1003000225
4,04177030908,1003000407
...,...,...
1101677,64914050829,1992999437
1101678,55003040118,1992999551
1101679,05606041341,1992999817
1101680,04102051845,1992999825


In [4]:
zip_query= \
    """ 
    SELECT DISTINCT 
    Z.ZIP,
    Z.CBSA
    FROM 
    AMAEDW.ZIP_CBSA_ZR Z
    WHERE  
    Z.CBSA = 37980
    AND
    Z.ACTIVE_IND = 'Y'
    """

In [7]:
mno = pd.read_excel('../../Data/Credentialling/Philadelphia_CBSA_Customer_Orders_Mar2020-Aug2020_Sample.xlsx')

In [35]:
# zips = pd.read_sql(con=AMAEDW, sql=zip_query)
zips[100:120]

,ZIP,CBSA
100,08344,37980
101,08347,37980
102,08505,37980
103,08511,37980
104,08515,37980
105,08518,37980
106,08554,37980
107,08562,37980
108,08640,37980
109,08641,37980


In [44]:
def fix_zip(zip_list):
    nums = []
    for num in zip_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 8:
            num = '0' + num
        elif len(num) == 7:
            num = '00' + num
        elif len(num) == 6:
            num = '000' + num
        elif len(num) == 4:
            num = '0' + num
        nums.append(num)
    return nums
XX['LOCATION_ZIP']=fix_zip(list(XX['LOCATION_ZIP']))

In [26]:
mno['ZIP']=[str(x)[0:5] for x in mno.LOCATION_ZIP]

In [27]:
pd.merge(mno, zips, on='ZIP')

,ORDER_DATE,PRODUCT_DESC,CUSTOMER_NAME,CUSTOMER_CATEGORY_DESC,LOCATION_ADDRESS_1,LOCATION_ADDRESS_2,SUITE,LOCATION_ZIP,LOCATION_CITY,LOCATION_STATE,NPI_ORGANIZATION,ME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,NPI_PHYSICIAN,ZIPCODE,ZIP,CBSA
0,2020-07-21,Full AMA Physician Profile,COOPER UNIVERSITY HOSPITAL,Hospital,1 COOPER PLZ,NaN,None,081031461,CAMDEN,NJ,1538300579,3305151245,SINHA,ATHENA,NaN,1285012443,08103,08103,37980
1,2020-08-03,Full AMA Physician Profile,COOPER UNIVERSITY HOSPITAL,Hospital,1 COOPER PLZ,NaN,None,081031461,CAMDEN,NJ,1538300579,4102132802,CAMPBELL,MATTHEW,JOSEPH,1104269869,08103,08103,37980
2,2020-06-30,Full AMA Physician Profile,COOPER UNIVERSITY HOSPITAL,Hospital,1 COOPER PLZ,NaN,None,081031461,CAMDEN,NJ,1538300579,1602140161,QUIMBY,ERNIKA,GLENISHA,1740607282,08103,08103,37980
3,2020-04-13,Full AMA Physician Profile,COOPER UNIVERSITY HOSPITAL,Hospital,1 COOPER PLZ,NaN,None,081031461,CAMDEN,NJ,1538300579,5606901099,LORBER,RICHARD,OWEN,1669565149,08103,08103,37980
4,2020-06-30,Full AMA Physician Profile,COOPER UNIVERSITY HOSPITAL,Hospital,1 COOPER PLZ,NaN,None,081031461,CAMDEN,NJ,1538300579,49522070156,GANDHI,APURVA,SHAILESH,1235418468,08103,08103,37980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1992,2020-04-21,Full AMA Physician Profile,FOX NURSING HOME CORP,Ambulatory Care,2644 BRISTOL ROAD,NaN,None,18976,WARRINGTON,PA,1558342329,3545980842,CORRIELUS,SANUL,JEAN,1205831716,18976,18976,37980
1993,2020-05-06,Reappointment AMA Physician Pr,FOX NURSING HOME CORP,Ambulatory Care,2644 BRISTOL ROAD,NaN,None,18976,WARRINGTON,PA,1558342329,4177991509,MOREHOUSE,TODD,C,1376644997,18976,18976,37980
1994,2020-07-16,Full AMA Physician Profile,"STADIUM CAMPUS AMBULATORY SURGICAL CENTER, LLC",Ambulatory Care,3340 S BROAD ST,NaN,None,191455802,PHILADELPHIA,PA,1679658710,3503101474,HAQQIE,NADIA,NaN,1962721720,19145,19145,37980
1995,2020-07-10,Full AMA Physician Profile,CAMCARE HEALTH CORPORATION,Hospital,1315 N DELAWARE ST,NaN,None,080661367,PAULSBORO,NJ,1538338090,49537140011,MATHANDA,REEMA,RAMESH,1518373471,08066,08066,37980


In [24]:
for row in mno.itertuples():
    if str(row.LOCATION_ZIP)[0:5] not in list(zips.ZIP):

Pandas(Index=1978, ORDER_DATE=Timestamp('2020-06-02 00:00:00'), PRODUCT_DESC='Full AMA Physician Profile', CUSTOMER_NAME='DREXEL UNIVERSITY', CUSTOMER_CATEGORY_DESC='Hospital', LOCATION_ADDRESS_1='153 BRODHEAD RD', LOCATION_ADDRESS_2=nan, SUITE='None', LOCATION_ZIP='180178931', LOCATION_CITY='BETHLEHEM', LOCATION_STATE='PA', NPI_ORGANIZATION=1659875284, ME=4101760346, LAST_NAME='DI SESA', FIRST_NAME='VERDI', MIDDLE_NAME='JOS', NPI_PHYSICIAN=1063475234)
Pandas(Index=1979, ORDER_DATE=Timestamp('2020-03-16 00:00:00'), PRODUCT_DESC='Reappointment AMA Physician Pr', CUSTOMER_NAME='DREXEL UNIVERSITY', CUSTOMER_CATEGORY_DESC='Hospital', LOCATION_ADDRESS_1='153 BRODHEAD RD', LOCATION_ADDRESS_2=nan, SUITE='None', LOCATION_ZIP='180178931', LOCATION_CITY='BETHLEHEM', LOCATION_STATE='PA', NPI_ORGANIZATION=1659875284, ME=4102132403, LAST_NAME='BASS', FIRST_NAME='MAYA', MIDDLE_NAME='ALEXA', NPI_PHYSICIAN=1376988279)
Pandas(Index=1980, ORDER_DATE=Timestamp('2020-03-16 00:00:00'), PRODUCT_DESC='Reappo

In [39]:
npi_query= \
    """ 
    SELECT DISTINCT 
    I.ME,
    K.KEY_VAL AS NPI_NUMBER
    FROM 
    AMAEDW.PARTY_KEY K, AMAEDW.PARTY_ID_TO_ME_VW I
    WHERE  
    K.KEY_TYPE_ID = 38
    AND
    K.ACTIVE_IND = 'Y'
    AND
    I.PARTY_ID = K.PARTY_ID
    """
me_npi = pd.read_sql(con=AMAEDW, sql=npi_query)
me_npi

,PARTY_ID,KEY_VAL
0,3380928,1528265030
1,3389036,1447466313
2,3412214,1447468301
3,3369823,1447468970
4,3424730,1447469382
...,...,...
1105694,15642788,1407483035
1105695,15642942,1215559919
1105696,15643031,1760003529
1105697,15643066,1922466671


In [55]:
res = pd.read_csv('../../Data/Credentialling/Residents_Jun2019-Dec2019.csv')
key = pd.read_csv('../../Data/Credentialling/product_key.csv')

In [56]:
res = pd.merge(res, party_npi, on='PARTY_ID')
res = pd.merge(res, key, right_on='PRODUCT_ID', left_on='ORDER_PRODUCT_ID')
res.to_csv('../../Data/Credentialling/Residents_Jun2019-Dec2019.csv', index=False)

In [41]:
phillz = pd.read_excel("../../Data/Credentialling/Philly_Ask.xlsx")
XX = pd.merge(phillz, party_npi, on='PARTY_ID')

In [43]:
XX = XX.drop_duplicates(['PRODUCT_DESC','ME','NPI_x'])

In [49]:
XX.sample(2000).to_excel('../../Data/Credentialling/Philadelphia_CBSA_Customer_Orders_Mar2020-Aug2020_Sample_20-10-15.xlsx', index=False)

In [47]:
XX['ME']=fix_me(XX.ME)

In [46]:
# ppd = pd.read_csv('../../Data/Scorecard/ppd_data_20200530.csv')
def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums
# ppd['ME']=fix_me(list(ppd['ME']))

In [48]:
XX

,PRODUCT_DESC,CUSTOMER_CATEGORY_DESC_x,NPI_x,NAME,OTHER_NAME,MAILING_ADDRESS,MAILING_ZIP,MAILING_CITY,MAILING_STATE,LOCATION_ADDRESS,LOCATION_ZIP,LOCATION_CITY,LOCATION_STATE,FULL_DT,ME,PARTY_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,KEY_VAL
0,Full AMA Physician Profile,Hospital,1750371522,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-03-17,00802860331,1926979,TOWERS,ADELE,E,1902872542
1,Full AMA Physician Profile,Hospital,1750371522,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-04-21,02405901377,2074722,GOSSELIN,EDWARD,MICHAEL,1821015330
2,Full AMA Physician Profile,Hospital,1750371522,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-07-03,02507861973,2005780,PLUSZCZYNSKI,RONALD,NaN,1952303919
3,Full AMA Physician Profile,Hospital,1750371522,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-03-13,03305953414,2251673,BRABSTON,TIMOTHY,BRIEN,1184607863
4,Full AMA Physician Profile,Hospital,1750371522,HOLY REDEEMER HEALTH SYSTEM,HOLY REDEEMER HOSPITAL & MEDICAL CENTER,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,1648 HUNTINGDON PIKE,190468001,MEADOWBROOK,PA,2020-04-15,03843011569,2300620,WYATT,FELICIE,GRETCHEN,1740370261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3238,Reappointment AMA Physician Pr,Ambulatory Care,1811932270,"PHILADELPHIA SURGI CENTER, INC.",None,9500 ROOSEVELT BLVD,191153926,PHILADELPHIA,PA,9500 ROOSEVELT BLVD,191153926,PHILADELPHIA,PA,2020-08-10,04102981681,2486366,GOTTLIEB,NEIL,BRUCE,1649250382
3239,Reappointment AMA Physician Pr,Ambulatory Care,1811932270,"PHILADELPHIA SURGI CENTER, INC.",None,9500 ROOSEVELT BLVD,191153926,PHILADELPHIA,PA,9500 ROOSEVELT BLVD,191153926,PHILADELPHIA,PA,2020-08-19,04177870591,2414468,GABAY,RAPHAEL,NaN,1700863743
3240,Reappointment AMA Physician Pr,Ambulatory Care,1740785096,GASTROINTESTINAL ENDOSCOPY CENTER LLC,None,2500 YORK RD STE 300,189291098,JAMISON,PA,1600 HORIZON DR STE 107,18914,CHALFONT,PA,2020-03-30,04113121390,5164598,PIERRE,MARJORIE,NaN,1033472154
3241,Reappointment AMA Physician Pr,Ambulatory Care,1144275900,MT LAUREL ENDOSCOPY CENTER L P,None,15000 MIDLANTIC DR,80541573,MOUNT LAUREL,NJ,15000 MIDLANTIC DR,080541573,MOUNT LAUREL,NJ,2020-03-06,03306860797,2351849,SALOWE,DAVID,ROSS,1588622740


In [31]:
ref = pd.read_csv("../../Data/Credentialling/New_Reference.csv")

In [25]:
good_zips = list(zips['ZIP'])

In [34]:
ref['MAILING_ZIP'] = [x[0:5] for x in ref.MAILING_ZIP]
ref['LOCATION_ZIP'] = [x[0:5] for x in ref.LOCATION_ZIP]
ref[(ref['LOCATION_ZIP'].isin(good_zips))|(ref['MAILING_ZIP'].isin(good_zips))]

,CUSTOMER_KEY,NAME,NPI,TAXONOMY,CATEGORY,SUBPART,MAILING_ADDRESS,MAILING_ZIP,MAILING_CITY,MAILING_STATE,LOCATION_ADDRESS,LOCATION_ZIP,LOCATION_CITY,LOCATION_STATE,UPDATED
380,54998,ALBERT EINSTEIN MEDICAL CENTER,1902805245,General Acute Care Hospital,Hospital,NO,5501 OLD YORK RD,19141,PHILA,PA,5501 OLD YORK RD,19141,PHILA,PA,2009-08-20
381,54998,ALBERT EINSTEIN MEDICAL CENTER,1871592816,Skilled Nursing Facility,Hospital,NO,5501 OLD YORK RD,19141,PHILA,PA,5501 OLD YORK RD,19141,PHILA,PA,2012-02-08
382,54998,ALBERT EINSTEIN MEDICAL CENTER,1174522114,General Acute Care Hospital,Hospital,NO,5501 OLD YORK RD,19141,PHILA,PA,5501 OLD YORK RD,19141,PHILA,PA,2008-06-02
383,54998,ALBERT EINSTEIN MEDICAL CENTER,1609832286,Psychiatry & Neurology Psychiatry,Hospital,NO,101 E OLNEY AVENUE,19120,PHILA,PA,1200 W TABOR RD,19141,PHILA,PA,2007-11-01
384,54998,ALBERT EINSTEIN MEDICAL CENTER,1639287311,Psychiatric Unit,Hospital,YES,5501 OLD YORK RD,19141,PHILADELPHIA,PA,5501 OLD YORK RD,19141,PHILADELPHIA,PA,2011-02-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18087,1006,THOMAS JEFFERSON UNIVERSITY HOSPITAL,1205207164,General Acute Care Hospital Critical Access,Hospital,NO,33 MEADOW CREEK LN,19343,GLENMOORE,PA,33 MEADOW CREEK LN,19343,GLENMOORE,PA,2015-10-16
18088,55789,THOMAS JEFFERSON UNIVERSITY HOSPITAL,1205207164,General Acute Care Hospital Critical Access,Hospital,NO,33 MEADOW CREEK LN,19343,GLENMOORE,PA,33 MEADOW CREEK LN,19343,GLENMOORE,PA,2015-10-16
18538,804,VIRTUA HEALTH INC,1619176443,General Acute Care Hospital,Hospital,YES,20 W STOW RD,08053,MARLTON,NJ,401 ROUTE 73 N,08053,MARLTON,NJ,2014-12-23
18539,60013,VIRTUA HEALTH INC,1619176443,General Acute Care Hospital,Hospital,YES,20 W STOW RD,08053,MARLTON,NJ,401 ROUTE 73 N,08053,MARLTON,NJ,2014-12-23


In [2]:
results = datamart.get_datamart_results((2,3,4,5,6,7),2020)

309332
145853
238404
145273


In [4]:
results.columns

Index(['FULL_DT', 'ME', 'PARTY_ID', 'ORDER_NBR', 'ORDER_PRODUCT_ID',
       'ORDER_PHYSICIAN_HIST_KEY', 'CUSTOMER_KEY', 'CUSTOMER_NBR',
       'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME', 'CUSTOMER_TYPE_DESC',
       'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC'],
      dtype='object')

In [6]:
add = pd.read_excel('../../Data/Credentialling/Org_Addresses.xlsx')

In [9]:
add['Customer Number']

0             13323
1             17736
2             18125
3             18760
4            486394
            ...    
11407       4007512
11408       4009224
11409       4013302
11410    2501871519
11411    3515911307
Name: Customer Number, Length: 11412, dtype: int64

In [10]:
results['CUSTOMER_NBR']

0         000002041391
1         000002041391
2         000002041391
3         000002041391
4         000002041391
              ...     
237581    000002083791
237582    000003234540
237583    000004013330
237584    000002088174
237585    000002896052
Name: CUSTOMER_NBR, Length: 237586, dtype: object

In [16]:
lala[['CUSTOMER_NAME','Company Name']]

,CUSTOMER_NAME,Company Name
0,Fort Duncan Regional Medical Center,Fort Duncan Regional Medical Center
1,Fort Duncan Regional Medical Center,Fort Duncan Regional Medical Center
2,Fort Duncan Regional Medical Center,Fort Duncan Regional Medical Center
3,Fort Duncan Regional Medical Center,Fort Duncan Regional Medical Center
4,Fort Duncan Regional Medical Center,Fort Duncan Regional Medical Center
...,...,...
237498,"Surgical Specialists, Asc","SURGICAL SPECIALISTS, ASC"
237499,Methodist Hospital,Methodist Hospital
237500,Williams Eye Institute,Williams Eye Institute
237501,Ozarks Community Hospital,OZARKS COMMUNITY HOSPITAL


In [20]:
cols = {'Customer Number':'CUSTOMER_NBR', 
       'Street 1ST':'addr_line_1', 'Street 2ND':'addr_line_2', 'Street 3RD':'addr_line_3', 'City':'addr_city', 'State':'addr_state', 'Zip Code':'addr_zip'}
add = add.rename(columns=cols)

In [21]:
add.columns

Index(['CUSTOMER_NBR', 'Company Name', 'Attention First', 'Attention Last',
       'addr_line_1', 'addr_line_2', 'addr_line_3', 'addr_city', 'addr_state',
       'addr_zip', 'Phone Number', 'Email Address'],
      dtype='object')

In [8]:
results.CUSTOMER_NBR = results.CUSTOMER_NBR.astype(float)
lala = pd.merge(results, addresses, left_on='CUSTOMER_NBR', right_on='Customer Number')
pd.merge(add, results, left_on='Customer Number', right_on='CUSTOMER_NBR')

ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [73]:
# ref['CUSTOMER_KEY']=ref['CUSTOMER_KEY'].astype(str)
mail = list(ref[(ref['MAILING_ZIP'].isin(good_zips))]['CUSTOMER_KEY'])
loc= list(ref[(ref['LOCATION_ZIP'].isin(good_zips))]['CUSTOMER_KEY'])
both= list(ref[(ref['MAILING_ZIP'].isin(good_zips))&(ref['LOCATION_ZIP'].isin(good_zips))]['CUSTOMER_KEY'])
either = list(ref[(ref['LOCATION_ZIP'].isin(good_zips))|(ref['MAILING_ZIP'].isin(good_zips))]['CUSTOMER_KEY'])

In [74]:
print(f"{len(CUSTOMER[CUSTOMER.CUSTOMER_KEY.isin(mail)])} mailing")
print(f"{len(CUSTOMER[CUSTOMER.CUSTOMER_KEY.isin(loc)])} location")
print(f"{len(CUSTOMER[CUSTOMER.CUSTOMER_KEY.isin(both)])} both")
print(f"{len(CUSTOMER[CUSTOMER.CUSTOMER_KEY.isin(either)])} either")

1154 mailing
1079 location
1023 both
1210 either


In [40]:
philly_place

[54998,
 54998,
 54998,
 54998,
 54998,
 54998,
 54998,
 54998,
 54998,
 54998,
 60774,
 60774,
 60774,
 60774,
 55677,
 55677,
 55677,
 55677,
 55677,
 55677,
 56137,
 132870,
 57081,
 57081,
 57081,
 129688,
 53509,
 129693,
 59926,
 59926,
 59926,
 59926,
 59926,
 59926,
 59926,
 59926,
 59926,
 59926,
 55465,
 55465,
 55465,
 55465,
 55465,
 55465,
 55322,
 71036,
 55582,
 56732,
 56732,
 56732,
 56732,
 56732,
 56732,
 56732,
 56732,
 56732,
 56732,
 53921,
 53921,
 53921,
 53921,
 53921,
 53921,
 53921,
 53921,
 58744,
 58744,
 58744,
 58744,
 58744,
 58744,
 58744,
 58744,
 60514,
 60514,
 62439,
 60051,
 60051,
 60587,
 60037,
 60037,
 60037,
 60037,
 60037,
 54289,
 54313,
 134405,
 134405,
 58915,
 58915,
 58915,
 58915,
 58915,
 58915,
 56394,
 56394,
 56394,
 52600,
 52600,
 52600,
 52600,
 52600,
 52600,
 52600,
 63680,
 59091,
 59091,
 59091,
 59091,
 59091,
 59091,
 51947,
 51947,
 51947,
 56927,
 56927,
 56927,
 56927,
 131674,
 55349,
 55349,
 54046,
 54046,
 54046,
 5

In [42]:
CUSTOMER['CUSTOMER_KEY'].asint

0        60362
1        59896
2        59896
3        59896
4        59896
         ...  
44317    56324
44318    52262
44319    56127
44320    70923
44321    59064
Name: CUSTOMER_KEY, Length: 44322, dtype: object

In [76]:
BEEP = ref[ref['MAILING_ZIP'].isin(good_zips)].sort_values('UPDATED').drop_duplicates('CUSTOMER_KEY', keep='last')

In [103]:
# BEEP_2 = pd.merge(CUSTOMER, BEEP, on='CUSTOMER_KEY')
JUNE = pd.merge(BEEP_2, ppd, on='ME', how = 'left')

In [111]:
bah[['FULL_DT', 'ME', 'KEY_VAL', 'CUSTOMER_NAME', 'NAME', 'NPI', 'MAILING_ADDRESS', 'MAILING_ZIP', 'MAILING_CITY',
       'MAILING_STATE', 'LOCATION_ADDRESS', 'LOCATION_ZIP', 'LOCATION_CITY',
       'LOCATION_STATE','LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'PRODUCT_DESC']].to_excel('../../Data/Credentialling/May-June_2020_Credentialing_Product.xlsx')

In [87]:
ppd.columns

Index(['ME', 'RECORD_ID', 'UPDATE_TYPE', 'ADDRESS_TYPE', 'MAILING_NAME',
       'LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX', 'MAILING_LINE_1',
       'MAILING_LINE_2', 'CITY', 'STATE', 'ZIP', 'SECTOR', 'CARRIER_ROUTE',
       'ADDRESS_UNDELIVERABLE_FLAG', 'FIPS_COUNTY', 'FIPS_STATE',
       'PRINTER_CONTROL_CODE', 'PC_ZIP', 'PC_SECTOR', 'DELIVERY_POINT_CODE',
       'CHECK_DIGIT', 'PRINTER_CONTROL_CODE_2', 'REGION', 'DIVISION', 'GROUP',
       'TRACT', 'SUFFIX_CENSUS', 'BLOCK_GROUP', 'MSA_POPULATION_SIZE',
       'MICRO_METRO_IND', 'CBSA', 'CBSA_DIV_IND', 'MD_DO_CODE', 'BIRTH_YEAR',
       'BIRTH_CITY', 'BIRTH_STATE', 'BIRTH_COUNTRY', 'GENDER',
       'TELEPHONE_NUMBER', 'PRESUMED_DEAD_FLAG', 'FAX_NUMBER', 'TOP_CD',
       'PE_CD', 'PRIM_SPEC_CD', 'SEC_SPEC_CD', 'MPA_CD', 'PRA_RECIPIENT',
       'PRA_EXP_DT', 'GME_CONF_FLG', 'FROM_DT', 'TO_DT', 'YEAR_IN_PROGRAM',
       'POST_GRADUATE_YEAR', 'GME_SPEC_1', 'GME_SPEC_2', 'TRAINING_TYPE',
       'GME_INST_STATE', 'GME_INST_ID', 'ME

In [102]:
BEEP_2 = pd.merge(BEEP_2, party_npi, on='PARTY_ID', how='left')

In [95]:
prod = pd.read_csv('../../Data/Credentialling/product_key.csv')

In [104]:
june = pd.merge(JUNE, prod, left_on='ORDER_PRODUCT_ID', right_on='PRODUCT_ID')

In [110]:
bah = june.sort_values('FULL_DT')[227:]

In [5]:
results

Index(['FULL_DT', 'ME', 'PARTY_ID', 'ORDER_NBR', 'ORDER_PRODUCT_ID',
       'ORDER_PHYSICIAN_HIST_KEY', 'CUSTOMER_KEY', 'CUSTOMER_NBR',
       'CUSTOMER_ISELL_LOGIN', 'CUSTOMER_NAME', 'CUSTOMER_TYPE_DESC',
       'CUSTOMER_TYPE', 'CUSTOMER_CATEGORY_DESC'],
      dtype='object')

In [7]:
customer = pd.read_csv('../../Data/Credentialling/Matched_Customers.csv')

In [43]:
addresses = pd.read_excel('../../Data/Credentialling/Org_Addresses.xlsx')

In [11]:
results.CUSTOMER_NBR = results.CUSTOMER_NBR.astype(float)
lala = pd.merge(results, add, left_on='CUSTOMER_NBR', right_on='Customer Number')

In [45]:
results.CUSTOMER_NBR = results.CUSTOMER_NBR.astype(float)
lala = pd.merge(results, addre_on='CUSTOMER_NBR', right_on='Customer Number')

In [47]:
hosps = lala[lala.CUSTOMER_CATEGORY_DESC.isin(['Hospital','Ambulatory Care'])]

In [50]:
hosps['Short_Zip'] = [str(x)[0:5] for x in hosps['Zip Code']]
philly_orders = hosps[hosps.Short_Zip.isin(good_zips)]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [42]:
addresses['Customer Number']

0             13323
1             17736
2             18125
3             18760
4            486394
            ...    
11407       4007512
11408       4009224
11409       4013302
11410    2501871519
11411    3515911307
Name: Customer Number, Length: 11412, dtype: object

In [46]:
lala

,FULL_DT,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,...,Attention First,Attention Last,Street 1ST,Street 2ND,Street 3RD,City,State,Zip Code,Phone Number,Email Address
0,2020-02-03,-1,-1,04599702,4915513,-1,52184,2041391.0,48HP62497,Fort Duncan Regional Medical Center,...,Romana,Besa,3333 N Foster Maldonado Blvd,NaN,NaN,Eagle Pass,TX,788525148,830 872-2751,ROMANA.BESA@UHSINC.COM
1,2020-02-03,-1,-1,04599703,4915513,-1,52184,2041391.0,48HP62497,Fort Duncan Regional Medical Center,...,Romana,Besa,3333 N Foster Maldonado Blvd,NaN,NaN,Eagle Pass,TX,788525148,830 872-2751,ROMANA.BESA@UHSINC.COM
2,2020-02-03,-1,-1,04599704,4915514,-1,52184,2041391.0,48HP62497,Fort Duncan Regional Medical Center,...,Romana,Besa,3333 N Foster Maldonado Blvd,NaN,NaN,Eagle Pass,TX,788525148,830 872-2751,ROMANA.BESA@UHSINC.COM
3,2020-02-21,01601920065,2102874,42839855,4915514,7748133,52184,2041391.0,48HP62497,Fort Duncan Regional Medical Center,...,Romana,Besa,3333 N Foster Maldonado Blvd,NaN,NaN,Eagle Pass,TX,788525148,830 872-2751,ROMANA.BESA@UHSINC.COM
4,2020-03-05,05104070177,3499150,43114034,4915513,8532430,52184,2041391.0,48HP62497,Fort Duncan Regional Medical Center,...,Romana,Besa,3333 N Foster Maldonado Blvd,NaN,NaN,Eagle Pass,TX,788525148,830 872-2751,ROMANA.BESA@UHSINC.COM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237498,2020-07-24,03005780101,2263132,43467346,4915513,11378533,54504,2099069.0,11AM71313,"Surgical Specialists, Asc",...,NaN,NaN,"Third Floor, Suite 300",1034 Mar Walt Dr,NaN,Fort Walton Beach,FL,325476639,850 315-9208,NaN
237499,2020-07-23,03548120577,5128193,43464386,4915513,11381506,54490,2083791.0,05HP00236,Methodist Hospital,...,Jill,Parker,300 W Huntington Dr,NaN,NaN,Arcadia,CA,910073473,626 574-3795,JILL.PARKER@METHODISTHOSPITAL.ORG
237500,2020-07-20,01602160463,6438105,43458886,4915513,11383637,57884,3234540.0,None,Williams Eye Institute,...,NaN,NaN,6850 Hohman Ave,NaN,NaN,Hammond,IN,463241410,NaN,NaN
237501,2020-07-24,04813111280,4903250,43467806,4915513,11390170,58715,2088174.0,28HP72503,Ozarks Community Hospital,...,NaN,NaN,2828 N National Ave,NaN,NaN,Springfield,MO,658034306,417 837-4004,NaN


In [53]:
len(philly_orders['CUSTOMER_NBR'].unique())

102

In [57]:
philly_orders.drop_duplicates(subset=['CUSTOMER_NBR'])[['CUSTOMER_NAME', 'CUSTOMER_CATEGORY_DESC', 'Customer Number',
       'Company Name', 'Attention First', 'Attention Last', 'Street 1ST',
       'Street 2ND', 'Street 3RD', 'City', 'State', 'Zip Code', 'Phone Number',
       'Email Address', 'Short_Zip']].to_csv('../../Data/Credentialling/Philly_Place.csv', index=False)